# Centrale 2021

In [1]:
type graphe = int list array ;; 

type graphe = int list array


In [2]:
let nombre_aretes g =
  let rec length_list l =
    match l with
    |[] -> 0
    |h::t -> 1 + length_list t 
  in
  let s = ref 0 in
  for i = 0 to Array.length(g) - 1 do
    s := !s + length_list g.(i)
  done;
  !s / 2 ;;

val nombre_aretes : 'a list array -> int = <fun>


In [3]:
let g_2_3 = [|
  [|3; 1|];
  [|4; 0; 2|];
  [|5; 1|];
  [|0; 4|];
  [|1; 3 ; 5|];
  [|2; 4|];
|]

val g_2_3 : int array array =
  [|[|3; 1|]; [|4; 0; 2|]; [|5; 1|]; [|0; 4|]; [|1; 3; 5|]; [|2; 4|]|]


In [4]:
let adjacence g =
let n = Array.length g in
let adj = Array.make n [||] in
for i = 0 to n - 1 do
    adj.(i) <- Array.of_list g.(i)
done;
adj ;; 

val adjacence : 'a list array -> 'a array array = <fun>


In [5]:
let test = [| []; [2]; [1]|]
let test_adj = adjacence test ;;

val test : int list array = [|[]; [2]; [1]|]


val test_adj : int array array = [|[||]; [|2|]; [|1|]|]


In [6]:
let rang (p, q) (s, t) =
  let is, js = s / p, s mod p in
  let it, jt = t / p, t mod p in
  if it = is + 1 then
    (q - 1) * is + js
  else if jt = js + 1 then
    p * (q - 1) + (p - 1) * js + is
  else 
    failwith "Argument(s) invalide(s)" ;;

val rang : int * int -> int * int -> int = <fun>


In [7]:
let test_rg = rang (4, 3) (2, 6)

val test_rg : int = 2


In [8]:
let sommets (p, q) rg =
    if rg < p * (q - 1) then
        let is, js = rg mod (q - 1), rg / (q - 1) in
        let s = is * p + js in
        (s, s + p)
    else if rg < p * (q - 1) + q * (p - 1) then
        let shift = p * (q - 1) in
        let is, js = (rg - shift) mod (p - 1), (rg - shift) / (p - 1) in
        let s = js * (q + 1) + is in
        (s, s + 1)
    else failwith "Argument(s) invalide(s)"

val sommets : int * int -> int -> int * int = <fun>


In [9]:
let test_sommets = sommets (4, 3) 16

val test_sommets : int * int = (10, 11)


In [10]:
let quadrillage p q =
  let graphe = Array.make (p * q) [] in
  let rec remplissage_graphe rg =
    if rg < p * (q - 1) + q * (p - 1) then
      let v1, v2 = sommets (p, q) rg in
      begin
        graphe.(v1) <- v2 :: graphe.(v1) ;
        graphe.(v2) <- v1 :: graphe.(v2);
        remplissage_graphe (rg + 1);
      end
  in
  remplissage_graphe 0 ;
  graphe ;;

val quadrillage : int -> int -> int list array = <fun>


In [11]:
let test_quadrillage = quadrillage 4 3 ;

val test_quadrillage : int list array =
  [|[1; 4]; [2; 0; 5]; [3; 1; 6]; [2; 7]; [5; 8; 0]; [6; 4; 9; 1];
    [7; 5; 10; 2]; [6; 11; 3]; [9; 4]; [10; 8; 5]; [11; 9; 6]; [10; 7]|]


In [12]:
type chemin = {debut : int; mutable fin : int ; suivant : int array}

type chemin = { debut : int; mutable fin : int; suivant : int array; }


In [13]:
let marche_aleatoire adj parent sommet =
    let chemin = {debut = sommet ; fin = sommet ; 
    suivant = Array.make (Array.length adj) 0} in
    while parent.(chemin.fin) = -2 do
        begin
        let nombre_voisins = Array.length adj.(chemin.fin) in
        let indice_voisin_aleatoire = Random.int nombre_voisins in
        let voisin_aleatoire = adj.(chemin.fin).(indice_voisin_aleatoire) in
        chemin.suivant.(chemin.fin) <- voisin_aleatoire; (*si u est dans le cycle, 
           cette modification n'importe pas*)
        chemin.fin <- voisin_aleatoire;
        end
    done;
    chemin ;;

val marche_aleatoire : int array array -> int array -> int -> chemin = <fun>


In [14]:
let greffe parent chemin =
    let sommet = ref chemin.debut in
    while !sommet <> chemin.fin do (*chemin.fin etant dans T, on a pas à l'ajouté*)
        let suivant = chemin.suivant.(!sommet) in
        parent.(!sommet) <- suivant;
        sommet := suivant
    done;

val greffe : int array -> chemin -> unit = <fun>


In [15]:
let wilson g r =
    let n = Array.length g in
    let adj = adjacence g in
    let parent = Array.make n (-2) in
    parent.(r) <- -1 ;
    for sommet = 0 to n - 1 do
        if parent.(sommet) < 0 then
            let chemin = marche_aleatoire adj parent sommet in
            greffe parent chemin
    done;
    parent ;;
    

val wilson : int list array -> int -> int array = <fun>


In [16]:
type direction = S | W | N | E

type direction = S | W | N | E


In [17]:
let p = 4
let q = 3
let g = quadrillage p q

val p : int = 4


val q : int = 3


val g : int list array =
  [|[1; 4]; [2; 0; 5]; [3; 1; 6]; [2; 7]; [5; 8; 0]; [6; 4; 9; 1];
    [7; 5; 10; 2]; [6; 11; 3]; [9; 4]; [10; 8; 5]; [11; 9; 6]; [10; 7]|]


In [18]:
let coord_noire sommet = 
  let i, j = sommet / p, sommet mod p in
  (i * 2, j * 2) 

val coord_noire : int -> int * int = <fun>


In [19]:
let test_coord = coord_noire 11

val test_coord : int * int = (4, 6)


In [20]:
let sommet_direction sommet direction =
    let i, j = sommet / p, sommet mod p in
    match direction with
    |N -> if i >= (q - 1) then -1 else (i + 1) * p + j
    |S -> if i <= 0 then -1 else (i - 1) * p + j
    |W -> if j <= 0 then -1 else i * p + j - 1
    |E -> if j >= (p - 1) then -1 else i * p + j + 1

val sommet_direction : int -> direction -> int = <fun>


In [21]:
let test_sommet_direction = sommet_direction 2 E

val test_sommet_direction : int = 3


In [22]:
let phi pavage = 
    let parent = Array.make (p * q) (-2) in
    parent.(0) <- -1;
    for i = 0 to q - 1 do
        for j = 0 to p - 1 do
            let sommet = i * p + j in
            let x_pavage, y_pavage = coord_noire sommet in
            if x_pavage mod 2 = y_pavage mod 2
                && (x_pavage, y_pavage) <> (0, 0) then
                let pere = sommet_direction sommet 
                                  pavage.(x_pavage).(y_pavage) in
                parent.(sommet) <- pere
          done;
    done;
    parent ;;

val phi : direction array array -> int array = <fun>


In [23]:
let pavage1 = [|
  [|N; N; E; N; N; N; W|];
  [|N; S; N; W; N; E; N|];
  [|S; N; W; N; W; N ;W|];
  [|N; N; N; N; N; W; N|];
  [|S; N; S; N; E; N; S|]
|] ;;

let test_phi = phi pavage1

val pavage1 : direction array array =
  [|[|N; N; E; N; N; N; W|]; [|N; S; N; W; N; E; N|];
    [|S; N; W; N; W; N; W|]; [|N; N; N; N; N; W; N|];
    [|S; N; S; N; E; N; S|]|]


val test_phi : int array = [|-1; 2; 6; 2; 0; 4; 5; 6; 4; 5; 11; 7|]


In [29]:
let rec representant partition sommet =
  if partition.(sommet) < 0 then sommet
  else representant partition partition.(sommet)

val representant : int array -> int -> int = <fun>


In [34]:
let union partition sommet1 sommet2 =
    let h_sommet1 = - partition.(sommet1) - 1 in
    let h_sommet2 = - partition.(sommet2) - 1 in
    if h_sommet1 > h_sommet2 then
        partition.(sommet2) <- sommet1
    else if h_sommet1 = h_sommet2 then
        begin
        partition.(sommet1) <- sommet2 ;
        partition.(sommet2) <- partition.(sommet2) - 1 
        end
    else
        partition.(sommet1) <- sommet2
    ;

val union : int array -> int -> int -> unit = <fun>


In [35]:
let count_composantes_connexes  graphe =
    let n = Array.length graphe in
    let count =  ref 0 in
    let partition = Array.make n (-1) in
    for sommet1 = 0 to n - 1 do
        List.iter (fun sommet2 ->
                  let representant_sommet1 = representant partition sommet1 in
                  let representant_sommet2 = representant partition sommet2 in
                  if representant_sommet1 <> representant_sommet2 then
                    union partition representant_sommet1 representant_sommet2)
                  graphe.(sommet1)
    done;
    for sommet = 0 to n - 1 do
        if partition.(sommet) < 0
            then incr count
    done;
    !count ;;
          

val count_composantes_connexes : int list array -> int = <fun>


In [36]:
let est_arbre graphe =
    let n = Array.length graphe in
    (nombre_aretes graphe = n - 1) && (count_composantes_connexes graphe) = 1

val est_arbre : int list array -> bool = <fun>
